In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sts

 # BS Option Pricing Function

In [2]:
class BS_euro_option :
    def call(S, K, T, r, sigma, div = 0):

        d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        call = (S * sts.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * sts.norm.cdf(d2, 0.0, 1.0))

        return call
    
    def put(S, K, T, r, sigma, div = 0):

        d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        put = (K * np.exp(-r * T) * sts.norm.cdf(-d2, 0.0, 1.0) - S * sts.norm.cdf(-d1, 0.0, 1.0))

        return put

 # Binomial Option Pricing Function

In [3]:
class Risk_neutral_u_d_p :
    
    ##########################################################################
    # You have to select The Model of Stock Movement or Dividend Processing ##
    ##########################################################################
    
    def no_dividends(sigma, T, N , Model = 'CRR', r= 0) :
        dt = T/N
        if Model.lower() == 'crr' :
            u = np.exp( sigma * np.sqrt(dt))
            d = 1/u
            
        elif Model.lower() == 'normal' :
            u = np.exp( r*dt + sigma * np.sqrt(dt))
            d = np.exp( r*dt - sigma * np.sqrt(dt))        

        else :
            u = np.exp( (r-0.5*sigma**2)*dt + sigma * np.sqrt(dt))
            d = np.exp( (r-0.5*sigma**2)*dt - sigma * np.sqrt(dt))

        p = (np.exp(r*dt)-d)/(u-d)
        return u,d,p
            
    def continuous_dividends(sigma, T, N, Model = 'CRR',div = 0, r=0) :
        dt = T/N
        if Model.lower() == 'crr' :
            u = np.exp( sigma * np.sqrt(dt))
            d = 1/u

        elif Model.lower() == 'normal' :
            u = np.exp( (r-div)*dt + sigma * np.sqrt(dt))
            d = np.exp( (r-div)*dt - sigma * np.sqrt(dt))

        else :
            u = np.exp( (r-div-0.5*sigma**2)*dt + sigma * np.sqrt(dt))
            d = np.exp( (r-div-0.5*sigma**2)*dt - sigma * np.sqrt(dt))
            
        p = (np.exp((r-div)*dt)-d)/(u-d)
        
        return u,d,p
    
    def Removing_nomlinearity(sigma,T,N,r , div = 0) :
        dt = T/N
        d1 = (np.log(S/K) + (r-div+0.5*sigma**2)*T) / (sigma * np.sqrt(T))
        d2 = (np.log(S/K) + (r-div-0.5*sigma**2)*T) / (sigma * np.sqrt(T))
        q_star = 0.5 + np.sign(d1) * np.sqrt( 0.25 - 0.25 * np.exp( - (d1/(N+1/3))**2 * (N+1/6)  )  ) 
        q = 0.5 + np.sign(d2) * np.sqrt( 0.25 - 0.25 * np.exp( - (d2/(N+1/3))**2 * (N+1/6)  )  ) 
        
        u = np.exp( (r-div)*dt  ) * q_star/q
        d = (np.exp(  (r-div)*dt ) - q*u)/(1-q)
        p = (np.exp((r-div)*dt)-d)/(u-d)
        return u, d, p
    
def make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = 'CRR',div = 0 , Removing_nonlinearity = False) :

    ####################################################
    ## This Function is used to make the Stock's Tree ##
    ####################################################
    
    dt = T/N
    if div>0 and Removing_nonlinearity == False :
        u,d,p = Risk_neutral_u_d_p.continuous_dividends(sigma = sigma , T = T , N = N , Model = Model , r = r, div = div)
    elif Removing_nonlinearity == True :
        u,d,p = Risk_neutral_u_d_p.Removing_nomlinearity(sigma = sigma , T = T , N = N  , r = r, div = div)
    else :
        u,d,p = Risk_neutral_u_d_p.no_dividends(sigma = sigma , T = T , N = N , Model = Model , r = r)

    Temp = np.ones((N+1,N+1))
    Temp[:,0] = 0
    Temp1 = Temp.cumsum(1)
    Te = Temp.copy()
    Te[0,:] = 0
    
    num_u = np.triu(Temp1 - Te.cumsum(0))
    num_d = np.triu(Temp1 - num_u)
    S_Tree = S * np.triu((u**num_u) * (d**num_d))
    return u,d,p ,S_Tree

In [4]:
class binomial_option :
    
    def Call(S,K,T,N,sigma,r , Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :        
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model, div= div , Removing_nonlinearity= Removing_nonlinearity)
        
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))

        C_exercise_Value = np.maximum( S_Tree - K , np.zeros(np.shape(S_Tree)))
        C = (C_exercise_Value[:,-1:] * sts.binom.pmf(B, N, p).round(7)).sum()  *  np.exp(-r*T)
        return C

    def Put(S,K,T,N,sigma,r , Model = 'CRR', div = 0, Removing_nonlinearity = False) :        
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model, div = div , Removing_nonlinearity=Removing_nonlinearity)

        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))

        P_exercise_Value = np.maximum( K- S_Tree , np.zeros(np.shape(S_Tree)))
        P = (P_exercise_Value[:,-1:] * sts.binom.pmf(B, N, p).round(7)).sum()  *  np.exp(-r*T)
        return P
    
    def Tree_C(S,K,T,N,sigma,r,Model = 'CRR', div = 0, For_American = False , Removing_nonlinearity = False) :
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model,div = div , Removing_nonlinearity=Removing_nonlinearity)
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))        
        C_exercise_Value = np.maximum( S_Tree - K , np.zeros(np.shape(S_Tree)))
        Temp_array= np.zeros(S_Tree.shape)
        dt = T/N
        
        if For_American == False :
            Temp_array[:,0] = C_exercise_Value[:,-1]
            for i in range(1,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        else :
            Temp_array[:,0] = C_exercise_Value[:,-1]
            Temp_array[:-1,1] = BS_euro_option.call(S = S_Tree[:,-2][:-1] , K = K , T = dt, sigma = sigma, r = r)
            for i in range(2,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        Option_Value_Tree = Temp_array[:,::-1]
        return Option_Value_Tree
    
    def Tree_P(S,K,T,N,sigma,r,Model = 'CRR', div = 0, For_American = False , Removing_nonlinearity = False) :
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model,div = div , Removing_nonlinearity= Removing_nonlinearity)
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))        
        P_exercise_Value = np.maximum(  K - S_Tree , np.zeros(np.shape(S_Tree)))
        Temp_array= np.zeros(S_Tree.shape)
        dt = T/N
        
        if For_American == False :
            Temp_array[:,0] = P_exercise_Value[:,-1]
            for i in range(1,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        else :
            Temp_array[:,0] = P_exercise_Value[:,-1]
            Temp_array[:-1,1] = BS_euro_option.put(S = S_Tree[:,-2][:-1] , K = K , T = dt, sigma = sigma, r = r)
            for i in range(2,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
            
        Option_Value_Tree = Temp_array[:,::-1]
        return Option_Value_Tree


 ## European

In [10]:
S = 101.17
K = 100
r = 0.01
sigma = 0.45
T = 1
N = 100
binomial_option.Put(S,K,T,N,sigma,r)

16.75017164728348

 ## American

In [11]:
binomial_option.Tree_P(S,K,T,N,sigma,r, For_American=True)[0,0]

16.769282438247092